In [1]:
import pandas as pd
rootpath='/home/mal/Documents/simra/Task2/'
d1=pd.read_csv(rootpath+'TRAIN-TASK2.csv')
d2=pd.read_csv(rootpath+'VALIDATION-TASK2.csv')
d3=pd.read_csv(rootpath+'TEST-TASK2.csv')
df = pd.concat([d1, d2], ignore_index=True)

In [2]:
df_new=df[:-1]

In [3]:
#preprocess labels
from ast import literal_eval
labels=[]
for id,row in df_new.iterrows():
    word = literal_eval(row['Y'])   
    labels.append(word)
    #break

In [4]:
#labels

In [5]:
#preprocess text
from ast import literal_eval
sents=[]
for id,row in df_new.iterrows():
    word = literal_eval(row['X'])   
    sents.append(word)
    #break

In [6]:
#sents

In [7]:
# #preprocess labels
# import re
# labels=[]
# for id,row in df_new.iterrows():
#     word = " ".join(re.findall("[a-zA-Z-]+",row['Y']))    
#     word =list(word.split(" "))
#     labels.append(word)
#     #break

In [8]:
sentences=[]
for sent in sents:
  #print(sent)
  temp=[]
  for sen in sent:
    #print(sen[0])
    temp.append(sen[0])
  sentences.append(temp)
  #print(sentences)
  #break

In [9]:
sentences[:10]

[['In',
  'late',
  'December',
  '2011',
  ',',
  'CrowdStrike',
  ',',
  'Inc.',
  'received',
  'three',
  'binary',
  'executable',
  'files',
  'that',
  'were',
  'suspected',
  'of',
  'having',
  'been',
  'involved',
  'in',
  'a',
  'sophisticted',
  'attack',
  'against',
  'a',
  'large',
  'Fortune',
  '500',
  'company',
  '.'],
 ['The',
  'files',
  'were',
  'analyzed',
  'to',
  'understand',
  'first',
  'if',
  'they',
  'were',
  'in',
  'fact',
  'malicious',
  ',',
  'and',
  'the',
  'level',
  'of',
  'sophistication',
  'of',
  'the',
  'samples',
  '.'],
 ['The',
  'samples',
  'were',
  'clearly',
  'malicious',
  'and',
  'varied',
  'in',
  'sophistication',
  '.'],
 ['All',
  'three',
  'samples',
  'provided',
  'remote',
  'access',
  'to',
  'the',
  'attacker',
  ',',
  'via',
  'two',
  'Command',
  'and',
  'Control',
  '(',
  'C2',
  ')',
  'servers',
  '.'],
 ['One',
  'sample',
  'is',
  'typical',
  'of',
  'what',
  'is',
  'commonly',
  'referr

In [10]:
new_sent=[]
new_lab=[]
for s,l in zip(sentences,labels):
  #print(s)
  #print(l)
  if(len(s)==len(l)):
    if(len(s)<=100):
        new_sent.append(s)
        new_lab.append(l)
    #continue
    #print('true')
    
  #break

In [11]:
sentences=new_sent
labels=new_lab

In [12]:
len(labels)

9472

In [13]:
sentences = [" ".join([s for s in sent]) for sent in sentences]

In [14]:
sentences[:10]

['In late December 2011 , CrowdStrike , Inc. received three binary executable files that were suspected of having been involved in a sophisticted attack against a large Fortune 500 company .',
 'The files were analyzed to understand first if they were in fact malicious , and the level of sophistication of the samples .',
 'The samples were clearly malicious and varied in sophistication .',
 'All three samples provided remote access to the attacker , via two Command and Control ( C2 ) servers .',
 "One sample is typical of what is commonly referred to as a 'dropper ' because its primary purpose is to write a malicious component to disk and connect it to the targeted hosts operating system .",
 'The malicious component in this case is what is commonly referred to as a Remote Access Tool ( RAT ) , this RAT is manifested as a Dynamic Link Library ( DLL ) installed as a service .',
 'The second sample analyzed is a dual use tool that can function both as a post exploitation tool used to inf

In [15]:
len(sentences[0])==len(labels[0])

False

In [16]:
#labels

In [17]:
lab=[]
for labl in labels:
  for lb in labl:
    lab.append(lb)

In [18]:
#lab
from collections import Counter 
counter = Counter(lab) 
print(counter) 

Counter({'O': 198409, 'I-Entity': 19854, 'B-Entity': 6863, 'B-Action': 3190, 'B-Modifier': 2002, 'I-Action': 773, 'I-Modifier': 136})


In [19]:
tags=dict(counter)

In [20]:
tags_vals=['B-Action','B-Entity','B-Modifier','I-Action','I-Entity','I-Modifier','O']
tag2idx={'B-Action': 0,
 'B-Entity': 1,
 'B-Modifier': 2,
 'I-Action': 3,
 'I-Entity': 4,
 'I-Modifier': 5,
 'O': 6}

In [21]:
!pip install pytorch-pretrained-bert
!pip install seqeval

In [51]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from tqdm import tqdm, trange

In [23]:
MAX_LEN = 128
bs =64

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [58]:
device2 = torch.device("cuda" )

In [59]:
torch.cuda.get_device_name(0) 

'TITAN RTX'

In [37]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)

The pre-trained model you are loading is an uncased model but you have set `do_lower_case` to False. We are setting `do_lower_case=True` for you but you may want to check this behavior.


In [38]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[0])

['in', 'late', 'december', '2011', ',', 'crowds', '##tri', '##ke', ',', 'inc', '.', 'received', 'three', 'binary', 'ex', '##ec', '##utable', 'files', 'that', 'were', 'suspected', 'of', 'having', 'been', 'involved', 'in', 'a', 'so', '##phi', '##stic', '##ted', 'attack', 'against', 'a', 'large', 'fortune', '500', 'company', '.']


In [39]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [40]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [41]:
len(input_ids)

9472

In [42]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [43]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [44]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [45]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [46]:
len(tag2idx)

7

In [47]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

In [52]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [53]:
from seqeval.metrics import f1_score
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [54]:
#input_ids_tensor = input_ids_tensor.to(self.device)
#segment_ids_tensor = segment_ids_tensor.to(self.device)
#input_mask_tensor = input_mask_tensor.to(self.device)

In [60]:
epochs = 2
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device2) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device2) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]


RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_index_select

In [43]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

RuntimeError: CUDA error: out of memory

In [0]:
#validation or test data
#preprocess labels
import re
labels=[]
for id,row in d3.iterrows():
    word = " ".join(re.findall("[a-zA-Z-]+",row['Y']))    
    word =list(word.split(" "))
    labels.append(word)
    #break

#preprocess text
from ast import literal_eval
sents=[]
for id,row in d3.iterrows():
    word = literal_eval(row['X'])   
    sents.append(word)
    #break

sentences=[]
for sent in sents:
    x=""
    for a,b in sent:
      x=x+' '+a
    sentences.append(x)
    #break
#print(sentences)

lab=[]
for labl in labels:
  for lb in labl:
    lab.append(lb)

#lab
from collections import Counter 
counter = Counter(lab) 
print(counter) 
tags=dict(counter)


Counter({'O': 12119, 'I-Entity': 499, 'B-Entity': 249, 'B-Action': 125, 'B-Modifier': 79, 'I-Action': 9})


In [0]:
tokenized_texts_test = [tokenizer.tokenize(sent) for sent in sentences]
input_ids_test = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_test],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
tagger_test = pad_sequences([[tags.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tags["O"], padding="post",
                     dtype="long", truncating="post")
attention_masks_test = [[float(i>0) for i in ii] for ii in input_ids_test]

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (597 > 512). Running this sequence through BERT will result in indexing errors


In [0]:
ts_inputs = torch.tensor(input_ids_test)
ts_tags = torch.tensor(tagger_test)
ts_masks = torch.tensor(attention_masks_test)
test_data = TensorDataset(ts_inputs, ts_masks, ts_tags)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=bs)

In [0]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tags))

100%|██████████| 407873900/407873900 [00:05<00:00, 71742473.39B/s]


In [0]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [0]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
epochs = 1
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

RuntimeError: ignored

In [0]:
# VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

In [63]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 5

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      logits = output[0]
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))



Epoch:   0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_index_select